In [2]:
import json
import ast

from pyspark.sql import Row
from pyspark.sql import SparkSession 


In [3]:
spark = SparkSession \
       .builder \
       .enableHiveSupport() \
       .getOrCreate()
#spark

## read from kafka

In [4]:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.156.0.3:6667,10.156.0.4:6667,10.156.0.5:6667,") \
  .option("subscribe", "mles.announcements") \
  .load() 

In [5]:
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
#df.printSchema()

## Handling data

In [6]:
rdd = df.select("value").rdd#.show()

In [7]:
#rdd.take(10)

In [8]:
rdd_json = rdd.map(lambda x: ast.literal_eval(x.asDict()['value']))

In [9]:
def cur_dict(**x):
    if len(x) < 14:
        r_key = ['lng', 'description', 'floorsCount', 'price', 'dateInserted', 'roomsCount', 
                 'priceType', 'status', 'address', 'category', 'floorNumber', 'announcementid', 
                 'lat', 'totalArea']
        key = list(x.keys())
        for i in r_key:
            if not (i in key) :
                x[i] = 'Null'
    x['dadd'] = x['dateInserted'][:10]
    return Row(**x)

In [10]:
rdd_row = rdd_json.map(lambda x: cur_dict(**x))

In [11]:
df2 = spark.createDataFrame(rdd_row)
#df2.collect()

## Write to hive

In [12]:
df2.write.format('orc').mode('overwrite').partitionBy('dadd').saveAsTable('sschokorov.test_pow')

In [13]:
#df2.printSchema()